In [1]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [ ]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.pickle')


In [2]:
subject_data, freqs, targets = load_psds('all_psds.pickle')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

### Logistic Regression

In [3]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets)
train_metric_dict, val_metric_dict

({'acc': 0.9515982055236226,
  'sens': 0.9606124306518724,
  'spec': 0.9410549528541012,
  'f1': 0.9553536790378895},
 {'acc': 0.7595682041216879,
  'sens': 0.7888349514563107,
  'spec': 0.7253371185237757,
  'f1': 0.7796101949025488})

In [5]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.9516040469180803,
  'sens': 0.9606232662968099,
  'spec': 0.9410549528541012,
  'f1': 0.9553593077325115},
 {'acc': 0.7595682041216879,
  'sens': 0.7888349514563107,
  'spec': 0.7253371185237757,
  'f1': 0.7796101949025488})

In [6]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [7]:
val_acc, train_acc

([0.7262021589793916,
  0.7340529931305201,
  0.7526987242394504,
  0.774288518155054,
  0.7392868825646058,
  0.7592410860320575,
  0.7579326136735361,
  0.7595682041216879],
 [0.7583765596523202,
  0.8085482966493761,
  0.8636910603299219,
  0.9034534324033834,
  0.9224613299686901,
  0.9415510070564045,
  0.9474099256974625,
  0.9516040469180803])

In [12]:
PCA_components = [30, 33, 35, 37, 40]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [13]:
val_acc, train_acc

([0.7713444553483808,
  0.7811579980372915,
  0.7870461236506379,
  0.7808308799476611,
  0.7733071638861629],
 [0.8950126174120286,
  0.8986225991868779,
  0.901642600121501,
  0.9020748633113697,
  0.903476797981214])

In [14]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9016951726716201,
  'sens': 0.9201846393897365,
  'spec': 0.8800694514853493,
  'f1': 0.9098477015958386},
 {'acc': 0.7841020608439647,
  'sens': 0.7979368932038835,
  'spec': 0.7679205110007097,
  'f1': 0.7993920972644377})

### k nearest neighbor classifier

In [ ]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 0)
train_metric_dict, val_metric_dict

In [ ]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 133)
train_metric_dict, val_metric_dict

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [ ]:
val_acc, train_acc

### Random Forest classifier

Here is a run of random Forest without doing PCA. As you can see in the definition of the method, PCA = 0, means we do not do PCA

In [ ]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 0)
train_metric_dict, val_metric_dict

Here is a run of random Forest with PCA. The maximum number of features is 133 here. 
As you can see, there is a significant reduction in validation set accuracy (from 75% to 71% )

In [ ]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 133)
train_metric_dict, val_metric_dict

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]